In [ ]:
## TO DO

# Fix the "Date" column

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [174]:
filterDay = ['yesterday','Today','Tomorrow']

filterHour = ['0','6','12','18']

urls = [f'https://www.santiago-airport.com/scl-departures?day={day}&tp={hour}' for day in filterDay for hour in filterHour]


In [211]:
urls

['https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0',
 'https://www.santiago-airport.com/scl-departures?day=yesterday&tp=6',
 'https://www.santiago-airport.com/scl-departures?day=yesterday&tp=12',
 'https://www.santiago-airport.com/scl-departures?day=yesterday&tp=18',
 'https://www.santiago-airport.com/scl-departures?day=Today&tp=0',
 'https://www.santiago-airport.com/scl-departures?day=Today&tp=6',
 'https://www.santiago-airport.com/scl-departures?day=Today&tp=12',
 'https://www.santiago-airport.com/scl-departures?day=Today&tp=18',
 'https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=0',
 'https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=6',
 'https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=12',
 'https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=18']

In [232]:
dataframes = []

for url in urls:
    
    request = requests.get(url)    
    soup = BeautifulSoup(request.text, 'html.parser')

    Reference_Day = url.split('=')[1][:-3]
    if Reference_Day == 'Tomorrow':
        Date = soup.find('option',{'value':'?day=tomorrow'}).text.split(' ')[0]
    else:
        Date = soup.find('option',{'selected':"selected"}).text.split(' ')[0]

    flights = soup.find_all('div', class_='flight-row')
    rows = []

    for flight in flights[1:]:
        row={}
        if flight.find('div',{'class','adsense'}): # advertisment row
            continue
        row['Destination'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('b').text
        row['Destination Code'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('span').text
        row['Departure'] = flight.find('div', {'class','flight-col flight-col__hour'}).text.strip()
        row['Flight'] = flight.find('div', {'class','flight-col flight-col__flight'}).text.strip()
        row['Airline'] = flight.find('div', {'class','flight-col flight-col__airline'}).text.strip()
        row['Terminal'] = flight.find('div', {'class','flight-col flight-col__terminal'}).text.strip()
        row['Status'] = flight.find_all('a')[-1].text
        row['Date'] = Date
        row['Reference Day'] = Reference_Day
        row['url'] = url
        rows.append(row)

    data = pd.DataFrame(rows)
    dataframes.append(data)

In [241]:
pd.options.display.max_rows = 50
pd.options.display.max_colwidth = None

In [242]:
pd.concat(dataframes)

,Destination,Destination Code,Departure,Flight,Airline,Terminal,Status,Date,Reference Day,url
0,Panama City,(PTY),00:26,CM118,Copa Airlines,2,Landed - On-time [+],2023-04-21,yesterday,https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0
1,Bogota,(BOG),00:30,LA572\nDL7387\nQF4207,LATAM Airlines\nDelta Air Lines\nQantas,2,Landed - Delayed [+],2023-04-21,yesterday,https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0
2,Auckland,(AKL),00:35,LA801\nCX7801\nMH9781\nQF3876,LATAM Airlines\nCathay Pacific\nMalaysia Airlines\nQantas,2,Landed - Delayed [+],2023-04-21,yesterday,https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0
3,Bogota,(BOG),00:40,AV240\nAC6153,Avianca\nAir Canada,2,Landed - On-time [+],2023-04-21,yesterday,https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0
4,Panama City,(PTY),00:47,CM276,Copa Airlines,2,Landed - On-time [+],2023-04-21,yesterday,https://www.santiago-airport.com/scl-departures?day=yesterday&tp=0
...,...,...,...,...,...,...,...,...,...,...
14,Antofagasta,(ANF),22:30,JA61,JetSMART,1,Scheduled [+],2023-04-23,Tomorrow,https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=18
15,Atlanta,(ATL),22:30,DL146\nKE7300\nLA6358,Delta Air Lines\nKorean Air\nLATAM Airlines,2,Scheduled - On-time [+],2023-04-23,Tomorrow,https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=18
16,New York,(JFK),22:55,LA532\nCX7822\nDL6061\nJL7627,LATAM Airlines\nCathay Pacific\nDelta Air Lines\nJAL Japan Airlines,2,Scheduled [+],2023-04-23,Tomorrow,https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=18
17,Miami,(MIA),23:20,LA500\nDL6056,LATAM Airlines\nDelta Air Lines,2,Scheduled [+],2023-04-23,Tomorrow,https://www.santiago-airport.com/scl-departures?day=Tomorrow&tp=18


In [209]:
request = requests.get(urls[0])

soup = BeautifulSoup(request.text, 'html.parser')

In [205]:
Date = soup.find('option',{'selected':"selected"}).text.split(' ')[0]
Reference_Day = soup.find('option',{'selected':"selected"}).text.split(' ')[-1]

In [206]:
flights = soup.find_all('div', class_='flight-row')

In [207]:
rows = []

for flight in flights[1:]:
    row={}
    if flight.find('div',{'class','adsense'}):
        continue
    row['Destination'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('b').text
    row['Destination Code'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('span').text
    row['Departure'] = flight.find('div', {'class','flight-col flight-col__hour'}).text.strip()
    row['Flight'] = flight.find('div', {'class','flight-col flight-col__flight'}).text.strip()
    row['Airline'] = flight.find('div', {'class','flight-col flight-col__airline'}).text.strip()
    row['Terminal'] = flight.find('div', {'class','flight-col flight-col__terminal'}).text.strip()
    row['Status'] = flight.find_all('a')[-1].text
    row['Date'] = Date
    row['Reference Day'] = Reference_Day
    rows.append(row)

In [208]:
data = pd.DataFrame(rows)

data

,Destination,Destination Code,Departure,Flight,Airline,Terminal,Status,Date,Reference Day
0,Panama City,(PTY),00:26,CM118,Copa Airlines,2,Landed - On-time [+],2023-04-21,Yesterday
1,Bogota,(BOG),00:30,LA572\nDL7387\nQF4207,LATAM Airlines\nDelta Air Lines\nQantas,2,Landed - Delayed [+],2023-04-21,Yesterday
2,Auckland,(AKL),00:35,LA801\nCX7801\nMH9781\nQF3876,LATAM Airlines\nCathay Pacific\nMalaysia Airli...,2,Landed - Delayed [+],2023-04-21,Yesterday
3,Bogota,(BOG),00:40,AV240\nAC6153,Avianca\nAir Canada,2,Landed - On-time [+],2023-04-21,Yesterday
4,Panama City,(PTY),00:47,CM276,Copa Airlines,2,Landed - On-time [+],2023-04-21,Yesterday
5,Buenos Aires,(EZE),00:50,H25577,Sky Airline,2,Landed - On-time [+],2023-04-21,Yesterday
6,Miami,(MIA),00:52,GG4530,Sky Lease Cargo,2,En Route [+],2023-04-21,Yesterday
7,Miami,(MIA),02:04,ET3806,Ethiopian Airlines,2,Landed - On-time [+],2023-04-21,Yesterday
8,Antofagasta,(ANF),02:20,LA1296,LATAM Airlines,1,En Route [+],2023-04-21,Yesterday
9,Panama City,(PTY),02:34,CM475,Copa Airlines,2,Landed - On-time [+],2023-04-21,Yesterday
